In [1]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

### Trajectories

In [2]:
T = 100
dt = 0.01
t = np.arange(T)
x = t*np.cos(5*np.pi*t/T)
y = t*np.sin(5*np.pi*t/T)
dx = np.append(np.array(0),(x[1:T] - x[0:(T-1)])/dt)
dy = np.append(np.array(0),(y[1:T] - y[0:(T-1)])/dt)
X = np.vstack((x,y,dx,dy))


### System Configurations

In [3]:
r = 100
R = r*np.identity(2)
H = np.hstack((np.identity(2), np.zeros((2,2))))


# 1.Naive Fusion

### Observation

In [4]:
S = 4
Z = np.zeros((2*S,T))
R_ = np.zeros((2*S,2*S))
for i in range(S):
    R_[i*2:i*2+2,i*2:i*2+2]  = R

Z = np.tile(np.dot(H,X), (S, 1)) + np.random.multivariate_normal(np.zeros(2*S), R_, T).T

## 1.1 Naive Fusion

In [5]:
F = np.array([[1,0,dt,0],[0,1,0,dt],[0,0,1,0],[0,0,0,1]])
Q = np.identity(4) * 100


X_k_k_s = np.tile(X[:,0], S).T + np.random.multivariate_normal(np.zeros(4*S), 100*np.identity(4*S))
P_k_k_s = np.tile(100 * np.identity(4),(S,1,1))

X_k_k_s_t = np.zeros((4*S,T))
X_k_k_t = np.zeros((4,T))
plt_X_k_k_s_t = np.zeros((T,4*S,T))
plt_X_k_k_t = np.zeros((T,4,T))

for t in range(T):
    for s in range(S):
        # Kalman Prediction
        X_k_kp = np.dot(F,X_k_k_s[4*s:4*(s+1)])
        P_k_kp = np.dot( np.dot(F, P_k_k_s[s,:,:]), F.T ) + Q

        # Kalman Filtering
        S_k = np.dot( np.dot(H, P_k_kp), H.T ) + R
        nu = Z[2*s:2*(s+1),t]-np.dot(H,X_k_kp)
        W_k_kp = np.dot( np.dot(P_k_kp, H.T), np.linalg.inv(S_k) ) 
        
        X_k_k_s[4*s:4*(s+1)] = X_k_kp + np.dot(W_k_kp, nu)
        P_k_k_s[s,:,:] = P_k_kp - np.dot( np.dot(W_k_kp, S_k), W_k_kp.T )
        X_k_k_s_t[4*s:4*(s+1),t] = X_k_k_s[4*s:4*(s+1)] 

    P_k_k = np.linalg.inv(sum([np.linalg.inv(P_k_k_s[s,:,:]) for s in range(S)]))
    X_k_k = np.dot(P_k_k, sum([np.dot(np.linalg.inv(P_k_k_s[s,:,:]), X_k_k_s[4*s:4*(s+1)]) for s in range(S)]))
    X_k_k_t[:,t] = X_k_k
    plt_X_k_k_s_t[t,:,:] = X_k_k_s_t
    plt_X_k_k_t[t,:,:] = X_k_k_t



### Display

In [6]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(figsize=(12, 12))

ax.set_xlim((-100, 100))
ax.set_ylim((-100, 100))

mark = [['gx','cx','mx','yx','kx','g:','c:','m:','y:','k:'],
        ['g--','c--','m--','y--','k--','g--','c--','m--','y--','k--']]


# lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]
lines = [ax.plot([], [], 'ro-',linewidth=3,ms=5)[0]] + \
    [ax.plot([], [], mark[0][i],linewidth=1,mew=2,ms=5)[0] for i in range(S)] + \
    [ax.plot([], [], mark[1][i],linewidth=1)[0] for i in range(S)] + \
    [ax.plot([], [], 'bo-',linewidth=3,markersize=5)[0]]

# initialization function: plot the background of each frame
def init():
    lines[0].set_data([], [])
    for i in range(S):
        lines[i+1].set_data([], [])
    for i in range(S):
        lines[S+i+1].set_data([], [])
    return lines
    lines[2*S+1].set_data([], [])

# animation function. This is called sequentially
def animate(t):
    lines[0].set_data(X[0,0:t], X[1,0:t])
    for s in range(S):
        lines[s+1].set_data(Z[2*s,0:t], Z[2*s+1,0:t])
    for s in range(S):
        lines[S+s+1].set_data(plt_X_k_k_s_t[t,4*s,0:t], plt_X_k_k_s_t[t,4*s+1,0:t])
    lines[2*S+1].set_data(plt_X_k_k_t[t,0,0:t], plt_X_k_k_t[t,1,0:t])
    return lines


# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=50, blit=True)

HTML(anim.to_html5_video())


### Error

In [7]:
print "S = ",S
print "Squred errors are"
print np.mean(np.square(X - X_k_k_t),axis=1)


S =  4
Squred errors are
[  2.87274189e+01   2.85239896e+01   4.09396882e+05   3.87742056e+05]


## 1.2 Naive Fusion with Retrodiction

In [8]:
F = np.array([[1,0,dt,0],[0,1,0,dt],[0,0,1,0],[0,0,0,1]])
Q = np.identity(4) * 100


X_k_k_s = np.tile(X[:,0], S).T + np.random.multivariate_normal(np.zeros(4*S), 100*np.identity(4*S))
P_k_k_s = np.tile(100 * np.identity(4),(S,1,1))

X_k_kp_s_t = np.zeros((4*S,T))
X_k_k_s_t = np.zeros((4*S,T))
P_k_kp_t_s = np.zeros((T,S,4,4))
P_k_k_t_s = np.zeros((T,S,4,4))

X_k_k_t = np.zeros((4,T))
plt_X_k_k_s_t = np.zeros((T,4*S,T))
plt_X_k_k_t = np.zeros((T,4,T))


for t in range(T):
    for s in range(S):
        # Kalman Prediction
        X_k_kp = np.dot(F,X_k_k_s[4*s:4*(s+1)])
        P_k_kp = np.dot( np.dot(F, P_k_k_s[s,:,:]), F.T ) + Q
        
        X_k_kp_s_t[4*s:4*(s+1),t] = X_k_kp
        P_k_kp_t_s[t,s,:,:] = P_k_kp
        
        # Kalman Filtering
        S_k = np.dot( np.dot(H, P_k_kp), H.T ) + R
        nu = Z[2*s:2*(s+1),t]-np.dot(H,X_k_kp)
        W_k_kp = np.dot( np.dot(P_k_kp, H.T), np.linalg.inv(S_k) ) 
        
        X_k_k_s[4*s:4*(s+1)] = X_k_kp + np.dot(W_k_kp, nu)
        P_k_k_s[s,:,:] = P_k_kp - np.dot( np.dot(W_k_kp, S_k), W_k_kp.T )
        X_k_k_s_t[4*s:4*(s+1),t] = X_k_k_s[4*s:4*(s+1)] 
        
        P_k_k_t_s[t,s,:,:] = P_k_k_s[s,:,:]
        
        for l in range(t-1,-1,-1):
            W_l_ln = np.dot( np.dot(P_k_k_t_s[l,s,:,:] , F.T) ,  np.linalg.inv(P_k_kp_t_s[l+1,s,:,:]))
            X_k_k_s_t[4*s:4*(s+1),l] = X_k_k_s_t[4*s:4*(s+1),l] + \
                np.dot(W_l_ln, (X_k_k_s_t[4*s:4*(s+1),l+1] - np.dot( F , X_k_k_s_t[4*s:4*(s+1),l] ) ) )
            P_k_k_t_s[l,s,:,:] = P_k_k_t_s[l,s,:,:] + \
                np.dot( np.dot( W_l_ln , ( P_k_k_t_s[l+1,s,:,:] - (np.dot( np.dot(F, P_k_k_t_s[l,s,:,:]), F.T ) + Q) ) ) , W_l_ln.T ) 
                
    for l in range(t,-1,-1):
        P_k_k = np.linalg.inv(sum([np.linalg.inv(P_k_k_t_s[l,s,:,:]) for s in range(S)]))
        X_k_k = np.dot(P_k_k, sum([np.dot(np.linalg.inv(P_k_k_t_s[l,s,:,:]), X_k_k_s_t[4*s:4*(s+1),l]) for s in range(S)]))
        X_k_k_t[:,l] = X_k_k
        
    plt_X_k_k_s_t[t,:,:] = X_k_k_s_t
    plt_X_k_k_t[t,:,:] = X_k_k_t



### Display

In [9]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(figsize=(12, 12))

ax.set_xlim((-100, 100))
ax.set_ylim((-100, 100))

mark = [['gx','cx','mx','yx','kx','g:','c:','m:','y:','k:'],
        ['g--','c--','m--','y--','k--','g--','c--','m--','y--','k--']]


# lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]
lines = [ax.plot([], [], 'ro-',linewidth=3,ms=5)[0]] + \
    [ax.plot([], [], mark[0][i],linewidth=1,mew=2,ms=5)[0] for i in range(S)] + \
    [ax.plot([], [], mark[1][i],linewidth=1)[0] for i in range(S)] + \
    [ax.plot([], [], 'bo-',linewidth=3,markersize=5)[0]]

# initialization function: plot the background of each frame
def init():
    lines[0].set_data([], [])
    for i in range(S):
        lines[i+1].set_data([], [])
    for i in range(S):
        lines[S+i+1].set_data([], [])
    return lines
    lines[2*S+1].set_data([], [])

# animation function. This is called sequentially
def animate(t):
    lines[0].set_data(X[0,0:t], X[1,0:t])
    for s in range(S):
        lines[s+1].set_data(Z[2*s,0:t], Z[2*s+1,0:t])
    for s in range(S):
        lines[S+s+1].set_data(plt_X_k_k_s_t[t,4*s,0:t], plt_X_k_k_s_t[t,4*s+1,0:t])
    lines[2*S+1].set_data(plt_X_k_k_t[t,0,0:t], plt_X_k_k_t[t,1,0:t])
    return lines


# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=50, blit=True)

HTML(anim.to_html5_video())


### Error

In [10]:
print "S = ",S
print "Squred errors are"
print np.mean(np.square(X - X_k_k_t),axis=1)


S =  4
Squred errors are
[  2.22702223e+02   2.27525809e+02   3.97161415e+05   3.80240941e+05]


# 2.Iterative Cross-Covariance

### Observation  (with dependency between sensors)

In [11]:
S = 6
Z = np.zeros((2*S,T))
R_ = np.zeros((2*S,2*S))
for i in range(S):
    R_[i*2:i*2+2,i*2:i*2+2]  = R
R_[0:2,2:4] = R
R_[2:4,0:2] = R
R_[0:2,4:6] = R
R_[4:6,0:2] = R
R_[2:4,4:6] = R
R_[4:6,2:4] = R

Z = np.tile(np.dot(H,X), (S, 1)) + np.random.multivariate_normal(np.zeros(2*S), R_, T).T

## 2.1 Naive Fusion

In [12]:
F = np.array([[1,0,dt,0],[0,1,0,dt],[0,0,1,0],[0,0,0,1]])
Q = np.identity(4) * 100


X_k_k_s = np.tile(X[:,0], S).T + np.random.multivariate_normal(np.zeros(4*S), 100*np.identity(4*S))
P_k_k_s = np.tile(100 * np.identity(4),(S,1,1))
P_star_k_k = np.eye(4*S)
H_star_k = np.tile(np.eye(4),(S,1))
X_star_k = np.zeros(4*S)

X_k_k_s_t = np.zeros((4*S,T))
X_k_k_t = np.zeros((4,T))
plt_X_k_k_s_t = np.zeros((T,4*S,T))
plt_X_k_k_t = np.zeros((T,4,T))

for t in range(T):
    for s in range(S):
        # Kalman Prediction
        X_k_kp = np.dot(F,X_k_k_s[4*s:4*(s+1)])
        P_k_kp = np.dot( np.dot(F, P_k_k_s[s,:,:]), F.T ) + Q

        # Kalman Filtering
        S_k = np.dot( np.dot(H, P_k_kp), H.T ) + R
        nu = Z[2*s:2*(s+1),t]-np.dot(H,X_k_kp)
        W_k_kp = np.dot( np.dot(P_k_kp, H.T), np.linalg.inv(S_k) ) 
        
        X_k_k_s[4*s:4*(s+1)] = X_k_kp + np.dot(W_k_kp, nu)
        P_k_k_s[s,:,:] = P_k_kp - np.dot( np.dot(W_k_kp, S_k), W_k_kp.T )
        X_k_k_s_t[4*s:4*(s+1),t] = X_k_k_s[4*s:4*(s+1)] 
        X_star_k[4*s:4*(s+1)] = X_k_k_s[4*s:4*(s+1)]
        P_star_k_k[4*s:4*(s+1),4*s:4*(s+1)] = P_k_k_s[s,:,:]
        
    P_k_k = np.linalg.inv(np.dot( np.dot( H_star_k.T , np.linalg.inv(P_star_k_k) ) , H_star_k) )
    X_k_k = np.dot(P_k_k ,  np.dot( np.dot( H_star_k.T , np.linalg.inv(P_star_k_k) ) , X_star_k) )
    X_k_k_t[:,t] = X_k_k
    plt_X_k_k_s_t[t,:,:] = X_k_k_s_t
    plt_X_k_k_t[t,:,:] = X_k_k_t



### Display

In [13]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(figsize=(12, 12))

ax.set_xlim((-100, 100))
ax.set_ylim((-100, 100))

mark = [['gx','cx','mx','yx','kx','g:','c:','m:','y:','k:'],
        ['g--','c--','m--','y--','k--','g--','c--','m--','y--','k--']]


# lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]
lines = [ax.plot([], [], 'ro-',linewidth=3,ms=5)[0]] + \
    [ax.plot([], [], mark[0][i],linewidth=1,mew=2,ms=5)[0] for i in range(S)] + \
    [ax.plot([], [], mark[1][i],linewidth=1)[0] for i in range(S)] + \
    [ax.plot([], [], 'bo-',linewidth=3,markersize=5)[0]]

# initialization function: plot the background of each frame
def init():
    lines[0].set_data([], [])
    for i in range(S):
        lines[i+1].set_data([], [])
    for i in range(S):
        lines[S+i+1].set_data([], [])
    return lines
    lines[2*S+1].set_data([], [])

# animation function. This is called sequentially
def animate(t):
    lines[0].set_data(X[0,0:t], X[1,0:t])
    for s in range(S):
        lines[s+1].set_data(Z[2*s,0:t], Z[2*s+1,0:t])
    for s in range(S):
        lines[S+s+1].set_data(plt_X_k_k_s_t[t,4*s,0:t], plt_X_k_k_s_t[t,4*s+1,0:t])
    lines[2*S+1].set_data(plt_X_k_k_t[t,0,0:t], plt_X_k_k_t[t,1,0:t])
    return lines


# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=50, blit=True)

HTML(anim.to_html5_video())


### Error

In [14]:
print "S = ",S
print "Squred errors are"
print np.mean(np.square(X - X_k_k_t),axis=1)


S =  6
Squred errors are
[  3.31623322e+01   2.85035722e+01   4.09519473e+05   3.87607503e+05]


## 2.2 Iterative Cross-Covariance

In [15]:
F = np.array([[1,0,dt,0],[0,1,0,dt],[0,0,1,0],[0,0,0,1]])
Q = np.identity(4) * 100


X_k_k_s = np.tile(X[:,0], S).T + np.random.multivariate_normal(np.zeros(4*S), 100*np.identity(4*S))
P_k_k_s = np.tile(100 * np.identity(4),(S,1,1))
P_star_k_k = np.eye(4*S)
H_star_k = np.tile(np.eye(4),(S,1))
X_star_k = np.zeros(4*S)

X_k_k_s_t = np.zeros((4*S,T))
X_k_k_t = np.zeros((4,T))
plt_X_k_k_s_t = np.zeros((T,4*S,T))
plt_X_k_k_t = np.zeros((T,4,T))
W_k_kp_s = np.zeros((S,4,2))

for t in range(T):
    for s in range(S):
        # Kalman Prediction
        X_k_kp = np.dot(F,X_k_k_s[4*s:4*(s+1)])
        P_k_kp = np.dot( np.dot(F, P_k_k_s[s,:,:]), F.T ) + Q

        # Kalman Filtering
        S_k = np.dot( np.dot(H, P_k_kp), H.T ) + R
        nu = Z[2*s:2*(s+1),t]-np.dot(H,X_k_kp)
        W_k_kp = np.dot( np.dot(P_k_kp, H.T), np.linalg.inv(S_k) ) 
        W_k_kp_s[s,:,:] = W_k_kp
        X_k_k_s[4*s:4*(s+1)] = X_k_kp + np.dot(W_k_kp, nu)
        P_k_k_s[s,:,:] = P_k_kp - np.dot( np.dot(W_k_kp, S_k), W_k_kp.T )
        X_k_k_s_t[4*s:4*(s+1),t] = X_k_k_s[4*s:4*(s+1)] 
        X_star_k[4*s:4*(s+1)] = X_k_k_s[4*s:4*(s+1)]
        P_star_k_k[4*s:4*(s+1),4*s:4*(s+1)] = P_k_k_s[s,:,:]
    
    for i in range(S):
        for j in range(S):
            if i==j:
                continue
            P_k_kp_ij = np.dot( np.dot(F , P_star_k_k[4*i:4*(i+1),4*j:4*(j+1)]) , F.T) + Q
            P_star_k_k[4*i:4*(i+1),4*j:4*(j+1)] = np.dot( np.dot( (np.eye(4) - np.dot( W_k_kp_s[i,:,:] , H)) , \
                                                                 P_k_kp_ij ) , \
                                                                (np.eye(4) - np.dot( W_k_kp_s[j,:,:] , H)).T )
    
            
    P_k_k = np.linalg.inv(np.dot( np.dot( H_star_k.T , np.linalg.inv(P_star_k_k) ) , H_star_k) )
    X_k_k = np.dot(P_k_k ,  np.dot( np.dot( H_star_k.T , np.linalg.inv(P_star_k_k) ) , X_star_k) )
    X_k_k_t[:,t] = X_k_k
    plt_X_k_k_s_t[t,:,:] = X_k_k_s_t
    plt_X_k_k_t[t,:,:] = X_k_k_t



### Display

In [16]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(figsize=(12, 12))

ax.set_xlim((-100, 100))
ax.set_ylim((-100, 100))

mark = [['gx','cx','mx','yx','kx','g:','c:','m:','y:','k:'],
        ['g--','c--','m--','y--','k--','g--','c--','m--','y--','k--']]


# lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]
lines = [ax.plot([], [], 'ro-',linewidth=3,ms=5)[0]] + \
    [ax.plot([], [], mark[0][i],linewidth=1,mew=2,ms=5)[0] for i in range(S)] + \
    [ax.plot([], [], mark[1][i],linewidth=1)[0] for i in range(S)] + \
    [ax.plot([], [], 'bo-',linewidth=3,markersize=5)[0]]

# initialization function: plot the background of each frame
def init():
    lines[0].set_data([], [])
    for i in range(S):
        lines[i+1].set_data([], [])
    for i in range(S):
        lines[S+i+1].set_data([], [])
    return lines
    lines[2*S+1].set_data([], [])

# animation function. This is called sequentially
def animate(t):
    lines[0].set_data(X[0,0:t], X[1,0:t])
    for s in range(S):
        lines[s+1].set_data(Z[2*s,0:t], Z[2*s+1,0:t])
    for s in range(S):
        lines[S+s+1].set_data(plt_X_k_k_s_t[t,4*s,0:t], plt_X_k_k_s_t[t,4*s+1,0:t])
    lines[2*S+1].set_data(plt_X_k_k_t[t,0,0:t], plt_X_k_k_t[t,1,0:t])
    return lines


# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=50, blit=True)

HTML(anim.to_html5_video())


### Error

In [17]:
print "S = ",S
print "Squred errors are"
print np.mean(np.square(X - X_k_k_t),axis=1)


S =  6
Squred errors are
[  3.31337564e+01   2.84410549e+01   4.09580255e+05   3.87685492e+05]


# 3.Distributed Kalman Filter

### Observation  

In [18]:
S = 6
Z = np.zeros((2*S,T))
R_ = np.zeros((2*S,2*S))
for i in range(S):
    R_[i*2:i*2+2,i*2:i*2+2]  = R

Z = np.tile(np.dot(H,X), (S, 1)) + np.random.multivariate_normal(np.zeros(2*S), R_, T).T

## 3.1 Naive Fusion (again)

In [19]:
F = np.array([[1,0,dt,0],[0,1,0,dt],[0,0,1,0],[0,0,0,1]])
Q = np.identity(4) * 100


X_k_k_s = np.tile(X[:,0], S).T + np.random.multivariate_normal(np.zeros(4*S), 100*np.identity(4*S))
P_k_k_s = np.tile(100 * np.identity(4),(S,1,1))
P_star_k_k = np.eye(4*S)
H_star_k = np.tile(np.eye(4),(S,1))
X_star_k = np.zeros(4*S)

X_k_k_s_t = np.zeros((4*S,T))
X_k_k_t = np.zeros((4,T))
plt_X_k_k_s_t = np.zeros((T,4*S,T))
plt_X_k_k_t = np.zeros((T,4,T))

for t in range(T):
    for s in range(S):
        # Kalman Prediction
        X_k_kp = np.dot(F,X_k_k_s[4*s:4*(s+1)])
        P_k_kp = np.dot( np.dot(F, P_k_k_s[s,:,:]), F.T ) + Q

        # Kalman Filtering
        S_k = np.dot( np.dot(H, P_k_kp), H.T ) + R
        nu = Z[2*s:2*(s+1),t]-np.dot(H,X_k_kp)
        W_k_kp = np.dot( np.dot(P_k_kp, H.T), np.linalg.inv(S_k) ) 
        
        X_k_k_s[4*s:4*(s+1)] = X_k_kp + np.dot(W_k_kp, nu)
        P_k_k_s[s,:,:] = P_k_kp - np.dot( np.dot(W_k_kp, S_k), W_k_kp.T )
        X_k_k_s_t[4*s:4*(s+1),t] = X_k_k_s[4*s:4*(s+1)] 
        X_star_k[4*s:4*(s+1)] = X_k_k_s[4*s:4*(s+1)]
        P_star_k_k[4*s:4*(s+1),4*s:4*(s+1)] = P_k_k_s[s,:,:]
        
    P_k_k = np.linalg.inv(np.dot( np.dot( H_star_k.T , np.linalg.inv(P_star_k_k) ) , H_star_k) )
    X_k_k = np.dot(P_k_k ,  np.dot( np.dot( H_star_k.T , np.linalg.inv(P_star_k_k) ) , X_star_k) )
    X_k_k_t[:,t] = X_k_k
    plt_X_k_k_s_t[t,:,:] = X_k_k_s_t
    plt_X_k_k_t[t,:,:] = X_k_k_t



### Display

In [20]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(figsize=(12, 12))

ax.set_xlim((-100, 100))
ax.set_ylim((-100, 100))

mark = [['gx','cx','mx','yx','kx','g:','c:','m:','y:','k:'],
        ['g--','c--','m--','y--','k--','g--','c--','m--','y--','k--']]


# lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]
lines = [ax.plot([], [], 'ro-',linewidth=3,ms=5)[0]] + \
    [ax.plot([], [], mark[0][i],linewidth=1,mew=2,ms=5)[0] for i in range(S)] + \
    [ax.plot([], [], mark[1][i],linewidth=1)[0] for i in range(S)] + \
    [ax.plot([], [], 'bo-',linewidth=3,markersize=5)[0]]

# initialization function: plot the background of each frame
def init():
    lines[0].set_data([], [])
    for i in range(S):
        lines[i+1].set_data([], [])
    for i in range(S):
        lines[S+i+1].set_data([], [])
    return lines
    lines[2*S+1].set_data([], [])

# animation function. This is called sequentially
def animate(t):
    lines[0].set_data(X[0,0:t], X[1,0:t])
    for s in range(S):
        lines[s+1].set_data(Z[2*s,0:t], Z[2*s+1,0:t])
    for s in range(S):
        lines[S+s+1].set_data(plt_X_k_k_s_t[t,4*s,0:t], plt_X_k_k_s_t[t,4*s+1,0:t])
    lines[2*S+1].set_data(plt_X_k_k_t[t,0,0:t], plt_X_k_k_t[t,1,0:t])
    return lines


# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=50, blit=True)

HTML(anim.to_html5_video())


### Error

In [21]:
print "S = ",S
print "Squred errors are"
print np.mean(np.square(X - X_k_k_t),axis=1)


S =  6
Squred errors are
[  2.03143401e+01   1.95878510e+01   4.08874472e+05   3.87779044e+05]


## 3.2 Distributed Kalman Filter

In [22]:
F = np.array([[1,0,dt,0],[0,1,0,dt],[0,0,1,0],[0,0,0,1]])
Q = np.identity(4) * 100


X_k_k_s = np.tile(X[:,0], S).T + np.random.multivariate_normal(np.zeros(4*S), 100*np.identity(4*S))
P_k_k_s = np.tile(100 * np.identity(4),(S,1,1))
P_star_k_k = np.eye(4*S)
H_star_k = np.tile(np.eye(4),(S,1))
X_star_k = np.zeros(4*S)

X_k_k_s_t = np.zeros((4*S,T))
X_k_k_t = np.zeros((4,T))
plt_X_k_k_s_t = np.zeros((T,4*S,T))
plt_X_k_k_t = np.zeros((T,4,T))
W_k_kp_s = np.zeros((S,4,2))
P_k_k = np.identity(4)
for t in range(T):
    P_k_k__ = S * P_k_k
    for s in range(S):
        # Kalman Prediction
        X_k_k__ = np.dot( np.dot( P_k_k__, np.linalg.inv(P_k_k_s[s,:,:]) ) , X_k_k_s[4*s:4*(s+1)] )
        
        X_k_kp = np.dot(F,X_k_k__)
        P_k_kp = np.dot( np.dot( F, P_k_k__ ), F.T ) + S * Q

        # Kalman Filtering
        S_k = np.dot( np.dot(H, P_k_kp), H.T ) + R
        nu = Z[2*s:2*(s+1),t]-np.dot(H,X_k_kp)
        W_k_kp = np.dot( np.dot(P_k_kp, H.T), np.linalg.inv(S_k) ) 
        W_k_kp_s[s,:,:] = W_k_kp
        X_k_k_s[4*s:4*(s+1)] = X_k_kp + np.dot(W_k_kp, nu)
        P_k_k_s[s,:,:] = P_k_kp - np.dot( np.dot(W_k_kp, S_k), W_k_kp.T )
        X_k_k_s_t[4*s:4*(s+1),t] = X_k_k_s[4*s:4*(s+1)] 
        X_star_k[4*s:4*(s+1)] = X_k_k_s[4*s:4*(s+1)]
        P_star_k_k[4*s:4*(s+1),4*s:4*(s+1)] = P_k_k_s[s,:,:]
    
    for i in range(S):
        for j in range(S):
            if i==j:
                continue
            P_k_kp_ij = np.dot( np.dot(F , P_star_k_k[4*i:4*(i+1),4*j:4*(j+1)]) , F.T) + Q
            P_star_k_k[4*i:4*(i+1),4*j:4*(j+1)] = np.dot( np.dot( (np.eye(4) - np.dot( W_k_kp_s[i,:,:] , H)) , \
                                                                 P_k_kp_ij ) , \
                                                                (np.eye(4) - np.dot( W_k_kp_s[j,:,:] , H)).T )
    
            
    P_k_k = np.linalg.inv(np.dot( np.dot( H_star_k.T , np.linalg.inv(P_star_k_k) ) , H_star_k) )
    X_k_k = np.dot(P_k_k ,  np.dot( np.dot( H_star_k.T , np.linalg.inv(P_star_k_k) ) , X_star_k) )
    X_k_k_t[:,t] = X_k_k
    plt_X_k_k_s_t[t,:,:] = X_k_k_s_t
    plt_X_k_k_t[t,:,:] = X_k_k_t



### Display

In [23]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(figsize=(12, 12))

ax.set_xlim((-100, 100))
ax.set_ylim((-100, 100))

mark = [['gx','cx','mx','yx','kx','g:','c:','m:','y:','k:'],
        ['g--','c--','m--','y--','k--','g--','c--','m--','y--','k--']]


# lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]
lines = [ax.plot([], [], 'ro-',linewidth=3,ms=5)[0]] + \
    [ax.plot([], [], mark[0][i],linewidth=1,mew=2,ms=5)[0] for i in range(S)] + \
    [ax.plot([], [], mark[1][i],linewidth=1)[0] for i in range(S)] + \
    [ax.plot([], [], 'bo-',linewidth=3,markersize=5)[0]]

# initialization function: plot the background of each frame
def init():
    lines[0].set_data([], [])
    for i in range(S):
        lines[i+1].set_data([], [])
    for i in range(S):
        lines[S+i+1].set_data([], [])
    return lines
    lines[2*S+1].set_data([], [])

# animation function. This is called sequentially
def animate(t):
    lines[0].set_data(X[0,0:t], X[1,0:t])
    for s in range(S):
        lines[s+1].set_data(Z[2*s,0:t], Z[2*s+1,0:t])
    for s in range(S):
        lines[S+s+1].set_data(plt_X_k_k_s_t[t,4*s,0:t], plt_X_k_k_s_t[t,4*s+1,0:t])
    lines[2*S+1].set_data(plt_X_k_k_t[t,0,0:t], plt_X_k_k_t[t,1,0:t])
    return lines


# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=50, blit=True)

HTML(anim.to_html5_video())


### Error

In [24]:
print "S = ",S
print "Squred errors are"
print np.mean(np.square(X - X_k_k_t),axis=1)


S =  6
Squred errors are
[  4.78632872e+01   3.78150899e+01   1.56926137e+07   1.43298556e+07]
